<a href="https://colab.research.google.com/github/WolfgangNS/Connect4/blob/main/connect_four_2024_v1_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Define functions
from sys import flags
import time
from IPython.display import clear_output
from enum import Flag, IntFlag, auto
import copy
from copy import deepcopy



def printc(text, color, end="\n"):
    color_codes = {
        "R": "\033[91m",
        "B": "\033[94m",
        "reset": "\033[0m"
    }
    print(color_codes.get(color, "") + text + color_codes["reset"], end=end)


class game:
  def __init__(self):
    self.board = [[], [], [], [], [], [], []] #each [] is a col, each [[]] is a row
    self.flagsB = [0b000000, 0b000000, 0b000000, 0b000000, 0b000000, 0b000000, 0b000000]
    self.flagsR = [0b000000, 0b000000, 0b000000, 0b000000, 0b000000, 0b000000, 0b000000]


  # def copy(self): #I could be more memory efficient by just listing moves
  #     new_instance = outcome(copy.copy(self))
  #     return new_instance

  def display(self):
    board = self.board

    clear_output(wait=True)
    for i in range(7)[::-1]: #each col
      for j in range(7): #each row
        try:
          printc("●", board[j][i], end="  ")
        except IndexError:
          print(" ", end="  ")
      print("\n", end="")
    time.sleep(0.2)


  def move(self, column, chip, display=True):
    board = self.board
    flagsB = self.flagsB
    flagsR = self.flagsR

    if(iswin(flagsB) or iswin(flagsR)):
      return #exit game if it's already won
    board[column].append(chip)
    if chip == "B":
      flagsB[column] = flagsB[column] | (1 << len(board[column])-1)
    else:
      flagsR[column] = flagsR[column] | (1 << len(board[column])-1)
    self.display() if display else None
    self.board = board
    self.flagsB = flagsB
    self.flagsR = flagsR

    if(iswin(flagsB) or iswin(flagsR)):
      print("\nWinner!") if display else None
      return

#test wins

def iswin(f1):
    #test diagonal wins
    f2 = []
    j = 0
    for i in f1:
        f2.append(i << j)
        j+=1

    for o in [0, 1, 2, 3]: # o for offset
      if(f2[o] & f2[o+1] & f2[o+2] & f2[o+3]):
        return True

    #Same, but opposite
    f2 = []
    j = 7
    for i in f1:
        f2.append(i << j)
        j-=1

    for o in [0, 1, 2, 3]: # o for offset
      if(f2[o] & f2[o+1] & f2[o+2] & f2[o+3]):
        return True

    #horizontal fours
    f2 = f1.copy()

    for o in [0, 1, 2, 3]: # o for offset
      if(f2[o] & f2[o+1] & f2[o+2] & f2[o+3]):
        return True

    #verticals
    for o in range(7): # o for offset
      if(f2[o] &
        f2[o] << 1 &
        f2[o] << 2 &
        f2[o] << 3):
        return True


#optimal move search

globalshortestwins = []
class outcome:
  def __init__(self, game, moves = [], depth=5, player="B", chip="B"):
    self.game = deepcopy(game)
    self.outcomes = {}
    self.iswin = iswin(self.game.flagsR) if player == "R" else iswin(self.game.flagsB)
    self.islose = iswin(self.game.flagsB) if player == "R" else iswin(self.game.flagsR)
    self.moves = moves.copy()
    self.shortestwins = []

    for i in range(7): #change this back to 7
      if not self.iswin and not self.islose and not depth==0:
        new_game = deepcopy(self.game)
        new_game.move(i, chip, False) #True for animation
        # print('blue win') if iswin(new_game.flagsB) else None

        new_moves = self.moves.copy()
        new_moves.append(i)

        nextchip = "R" if chip == "B" else "B" #this could be calculated from depth and modulo
        new_outcome = outcome(new_game, new_moves, depth-1, player, nextchip)
        if new_outcome.iswin:
          # self.shortestwins.append(new_moves)
          #to-do: this needs to report up the hierarchy somehow, to the root node
          globalshortestwins.append(new_moves)
          # print(new_moves)

        self.outcomes[i] = new_outcome
        #I wish I knew the difference between using self and not, e.g. self.game vs. game
        #I think it has to do with input argument vs. instance

        #ask ChatGPT to see if there's a way to batch process all possible moves in a faster way (GPU? matrix operation?)

In [2]:
#@title Randomized moves (no strategy)

import random

random.seed(911)
newgame = game()
moves = []
for i in range(50):
  chip = "R" if i % 2 == 0 else "B"
  r = random.randint(0,6)
  newgame.move(r, chip)
  moves.append(r)

   ●                 
   ●     ●     ●     
   ●     ●     ●     
●  ●     ●     ●     
●  ●  ●  ●     ●  ●  
●  ●  ●  ●  ●  ●  ●  
●  ●  ●  ●  ●  ●  ●  

Winner!


In [3]:
#@title Hyper defensive (block immediate losses for blue only)

import random

random.seed(69420)
newgame = game()
moves = []
for i in range(50):
  chip = "R" if i % 2 == 0 else "B"
  if chip == "R":
    globalshortestwins = []
    root = outcome(newgame, player="R", chip="R")
    try:
      shortest_win = min(globalshortestwins, key=len)
      r = shortest_win[0]
    except:
      r = random.randint(0,6)


  else: #chip is "B"
    globalshortestwins = []
    root = outcome(newgame, player="B", chip="B")
    try:
      shortest_win = min(globalshortestwins, key=len)
      r = shortest_win[0]
    except:
      r = random.randint(0,6)

    #blocking opponents first (overrides prior decision)
    globalshortestwins = [] #actually losses
    root = outcome(newgame, depth=1, player="R", chip="R") #this is weird because it imagines two plays in a row
    try:
      shortest_lose = min(globalshortestwins, key=len) #actually losses
      r = shortest_lose[0]
    except:
      r = random.randint(0,6)

  newgame.move(r, chip)
  moves.append(r)

                     
                     
   ●     ●           
●  ●  ●  ●           
●  ●  ●  ●           
●  ●  ●  ●  ●  ●     
●  ●  ●  ●  ●  ●  ●  

Winner!


In [4]:
#@title Hyper defensive for both players

import random

random.seed(42069)
newgame = game()
moves = []
for i in range(50):
  chip = "R" if i % 2 == 0 else "B"
  if chip == "R":
    globalshortestwins = []
    root = outcome(newgame, player="R", chip="R") #5 moves deep default
    try:
      shortest_win = min(globalshortestwins, key=len)
      r = shortest_win[0]
    except:
      r = random.randint(0,6)

    #blocking opponents first (overrides prior decision)
    globalshortestwins = [] #actually losses
    root = outcome(newgame, depth=1, player="B", chip="B") #this is weird because it imagines two plays in a row
    try:
      shortest_lose = min(globalshortestwins, key=len) #actually losses
      r = shortest_lose[0]
    except:
      r = random.randint(0,6)


  else: #chip is "B"
    globalshortestwins = []
    root = outcome(newgame, player="B", chip="B")
    try:
      shortest_win = min(globalshortestwins, key=len)
      r = shortest_win[0]
    except:
      r = random.randint(0,6)

    #blocking opponents first (overrides prior decision)
    globalshortestwins = [] #actually losses
    root = outcome(newgame, depth=1, player="R", chip="R") #this is weird because it imagines two plays in a row
    try:
      shortest_lose = min(globalshortestwins, key=len) #actually losses
      r = shortest_lose[0]
    except:
      r = random.randint(0,6)

  newgame.move(r, chip)
  moves.append(r)

   ●  ●  ●           
   ●  ●  ●  ●        
   ●  ●  ●  ●        
   ●  ●  ●  ●  ●  ●  
●  ●  ●  ●  ●  ●  ●  
●  ●  ●  ●  ●  ●  ●  
●  ●  ●  ●  ●  ●  ●  

Winner!


In [5]:
#@title Hyper defensive for both players (seed=1234)

import random

random.seed(1234)
newgame = game()
moves = []
for i in range(50):
  chip = "R" if i % 2 == 0 else "B"
  if chip == "R":
    globalshortestwins = []
    root = outcome(newgame, player="R", chip="R")
    try:
      shortest_win = min(globalshortestwins, key=len)
      r = shortest_win[0]
    except:
      r = random.randint(0,6)

    #blocking opponents first (overrides prior decision)
    globalshortestwins = [] #actually losses
    root = outcome(newgame, depth=1, player="B", chip="B") #this is weird because it imagines two plays in a row
    try:
      shortest_lose = min(globalshortestwins, key=len) #actually losses
      r = shortest_lose[0]
    except:
      r = random.randint(0,6)


  else: #chip is "B"
    globalshortestwins = []
    root = outcome(newgame, player="B", chip="B")
    try:
      shortest_win = min(globalshortestwins, key=len)
      r = shortest_win[0]
    except:
      r = random.randint(0,6)

    #blocking opponents first (overrides prior decision)
    globalshortestwins = [] #actually losses
    root = outcome(newgame, depth=1, player="R", chip="R") #this is weird because it imagines two plays in a row
    try:
      shortest_lose = min(globalshortestwins, key=len) #actually losses
      r = shortest_lose[0]
    except:
      r = random.randint(0,6)

  newgame.move(r, chip)
  moves.append(r)

●        ●  ●  ●     
●        ●  ●  ●     
●  ●     ●  ●  ●  ●  
●  ●     ●  ●  ●  ●  
●  ●     ●  ●  ●  ●  
●  ●  ●  ●  ●  ●  ●  
●  ●  ●  ●  ●  ●  ●  


In [8]:
#@title Long-term win strat for both players

import random

random.seed(69420)
newgame = game()
moves = []
for i in range(50):
  chip = "R" if i % 2 == 0 else "B"
  if chip == "R":
    globalshortestwins = []
    root = outcome(newgame, player="R", chip="R") #5 moves deep default
    try:
      shortest_win = min(globalshortestwins, key=len)
      r = shortest_win[0]
    except:
      r = random.randint(0,6)

    #blocking opponents first (overrides prior decision)
    globalshortestwins = [] #actually losses
    root = outcome(newgame, depth=1, player="B", chip="B")
    try:
      shortest_lose = min(globalshortestwins, key=len) #actually losses
      r = shortest_lose[0]
    except: None


  else: #chip is "B"
    globalshortestwins = []
    root = outcome(newgame, player="B", chip="B")
    try:
      shortest_win = min(globalshortestwins, key=len)
      r = shortest_win[0]
    except:
      r = random.randint(0,6)

    #blocking opponents first (overrides prior decision)
    globalshortestwins = [] #actually losses
    root = outcome(newgame, depth=1, player="R", chip="R") #this is weird because it imagines two plays in a row
    #^check to see whether this blocks immediate losses (or if I should change depth #)
    try:
      shortest_lose = min(globalshortestwins, key=len) #actually losses
      r = shortest_lose[0]
    except: None

  newgame.move(r, chip)
  moves.append(r)

                     
                     
●                    
●  ●     ●     ●     
●  ●  ●  ●     ●     
●  ●  ●  ●     ●     
●  ●  ●  ●     ●     

Winner!
